In [1]:
## Univariate Selection
# Is when we analisis each feature individualy to determiante the strength of the 
# relationship of that feature to the target value

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
diabetes_data = pd.read_csv('datasets/diabetes.csv')

diabetes_data.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [4]:
def get_selected_features(X, X_new):
    selected_features = []
    
    for i in range(len(X_new.columns)):
        for j in range(len(X.columns)):
            
            if (X_new.iloc[:, i].equals(X.iloc[:, j])):
                print(X.columns[j])
                selected_features.append(X.columns[j])
    
    return selected_features

In [5]:
# measure of dependency between variables -> use this
# relationship to determiante the most relevant features for classification
from sklearn.feature_selection import chi2

from sklearn.feature_selection import SelectKBest

In [6]:
X = diabetes_data.drop('Outcome', axis=1)
Y = diabetes_data['Outcome']

In [7]:
X.shape

(768, 8)

In [8]:
X = X.astype(np.float64)

In [9]:
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, Y)

In [10]:
fit.scores_

array([ 111.51969064, 1411.88704064,   17.60537322,   53.10803984,
       2175.56527292,  127.66934333,    5.39268155,  181.30368904])

In [13]:
feature_score = pd.DataFrame()

for i in range(X.shape[1]):
    new = pd.DataFrame({'Feature_name': X.columns[i],
                        'Score': fit.scores_[i]}, index=[i])
    
    feature_score = pd.concat([feature_score, new])

In [14]:
feature_score

,Feature_name,Score
0,Pregnancies,111.519691
1,Glucose,1411.887041
2,BloodPressure,17.605373
3,SkinThickness,53.108040
4,Insulin,2175.565273
5,BMI,127.669343
6,DiabetesPedigreeFunction,5.392682
7,Age,181.303689


In [15]:
X_new = fit.transform(X)

In [16]:
X_new = pd.DataFrame(X_new)

X_new.head()

,0,1,2,3
0,148.0,0.0,33.6,50.0
1,85.0,0.0,26.6,31.0
2,183.0,0.0,23.3,32.0
3,89.0,94.0,28.1,21.0
4,137.0,168.0,43.1,33.0


In [17]:
selected_feature = get_selected_features(X, X_new)

selected_feature

Glucose
Insulin
BMI
Age


['Glucose', 'Insulin', 'BMI', 'Age']

In [18]:
X[selected_feature].head()

,Glucose,Insulin,BMI,Age
0,148.0,0.0,33.6,50.0
1,85.0,0.0,26.6,31.0
2,183.0,0.0,23.3,32.0
3,89.0,94.0,28.1,21.0
4,137.0,168.0,43.1,33.0


In [19]:
chi2_best_feature = X[selected_feature]

In [21]:
# use the ANOVA F-value as a measure of dependency 
# use this relationship to determine the most relevant 
# features for classification
from sklearn.feature_selection import f_classif, SelectPercentile

test = SelectPercentile(f_classif, percentile=80)

fit = test.fit(X, Y)

In [22]:
fit.scores_

array([ 39.67022739, 213.16175218,   3.2569504 ,   4.30438091,
        13.28110753,  71.7720721 ,  23.8713002 ,  46.14061124])

In [23]:
X_new = fit.transform(X)

In [24]:
X_new = pd.DataFrame(X_new)

X_new.head()

,0,1,2,3,4,5
0,6.0,148.0,0.0,33.6,0.627,50.0
1,1.0,85.0,0.0,26.6,0.351,31.0
2,8.0,183.0,0.0,23.3,0.672,32.0
3,1.0,89.0,94.0,28.1,0.167,21.0
4,0.0,137.0,168.0,43.1,2.288,33.0


In [25]:
selected_feature = get_selected_features(X, X_new)

selected_feature

Pregnancies
Glucose
Insulin
BMI
DiabetesPedigreeFunction
Age


['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

In [26]:
f_classif_best_feature = X[selected_feature]

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [32]:
def build_model(X, Y, test_frac):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=test_frac)
    
    model = LogisticRegression(solver='liblinear').fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("Test Score: ", accuracy_score(y_test, y_pred))

In [33]:
build_model(X, Y, 0.2)

Test Score:  0.7662337662337663


In [34]:
build_model(chi2_best_feature, Y, 0.2)

Test Score:  0.7532467532467533


In [35]:
build_model(f_classif_best_feature, Y, 0.2)

Test Score:  0.7467532467532467
